# VIX Predictor using an adaBoost Model

In [51]:
# Import appropriate modules

import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import datetime
import numpy as np
import yfinance as yf
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from vix_functions import garch_fit_and_predict, correlation_filter, retrieve_yahoo_close, retrieve_yahoo_volume 

from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [407]:
threshold= 0.00 # 0.030
adaboost_estimators = 22
training_period_months = 120
# Inclusion of the first 4 components lag1
#n: number of components to include
number_of_pca_lag_component_to_include = 4
num_pca_components = 40
demo_mode = True
parameter_tuning_mode = False
run_multiple_tuning_iterations = False

# Generation of the Features Matrix X

In [408]:
# Ticker List: VIX must be in first position
ticker_list= ["^VIX", "spy", 'XLF', 'XLE',
              'EURUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'BRLUSD=X', "DX-Y.NYB","USDJPY=X", 
              '^TNX', 'ZB=F', 'ZF=F', 'NQ=F','NKD=F',
              'LQD',
              'AAPL', 'AMZN', 'GE','MU','MSFT', 'BMY', 'FDX', 'GS','PLD','NVDA',   
              "tlt", "ief", 
              "FXI", "EZU", "EEM", "EFA", 'FEZ', "^GDAXI", '^FTSE','^HSI','^FCHI',
              "gld", "slv", "CL=F"]
    
#
# Energy Select Sector SPDR Fund (XLE)
# Financial Select Sector SPDR Fund (XLF)
# CAC 40 (^FCHI)
# Yen Denominated TOPIX Futures,D (TPY=F)
# HANG SENG INDEX (^HSI)
# FTSE 100 (^FTSE)
# DAX PERFORMANCE-INDEX (^GDAXI)
# SPDR EURO STOXX 50 ETF (FEZ)
# Nikkei/USD Futures,Dec-2021 (NKD=F)
# Nasdaq 100 Dec 21 (NQ=F)
# NVIDIA Corporation (NVDA)
# 'WTI CRUDE FUTURE' '261220.KS' -- eliminate
# Euro spot  'EURUSD'
# Treasury Yield 10 Years (^TNX) -- 1985
# U.S. Treasury Bond Futures,Dec- (ZB=F) - 2000
# Five-Year US Treasury Note Futu (ZF=F) - 2000
# American Funds U.S. Government Securities Fund Class C (UGSCX) - 2001
# PIMCO High Yield Municipal Bond Fund Class A (PYMAX) --eliminate
# Vanguard High-Yield Corporate Fund Investor Shares (VWEHX) -- 1985 --eliminate
# iShares iBoxx $ Investment Grade Corporate Bond ETF (LQD)
# 13 Week Treasury Bill (^IRX) --eliminate
# Micron Technology, Inc. (MU)
# Microsoft Corporation (MSFT)
# Bristol-Myers Squibb Company (BMY)
# FEDEX CORP (FDX)
# The Goldman Sachs Group, Inc. (GS)
# Prologis, Inc. (PLD)


In [409]:
# Inclusion of security levels X1
def retrieve_close(close_prices_dict, ticker_list):
    for ticker in ticker_list:
        close_price = retrieve_yahoo_close(ticker, start_date='2006-07-02', end_date='2021-10-02')
        close_prices_dict[ticker] = close_price
    return close_prices_dict


if demo_mode == True:
    close_prices_df = pd.read_csv("adaboost_close_prices.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
else:
    close_prices_dict = {}
    close_prices_dict = retrieve_close(close_prices_dict, ticker_list)
    close_prices_df= pd.DataFrame(close_prices_dict)
    close_prices_df.to_csv("adaboost_close_prices.csv", index=True)
print("Completed retrieve of close prices")

Completed retrieve of close prices


In [410]:
close_prices_df=close_prices_df.ffill(axis='rows')
close_prices_df=close_prices_df.bfill(axis='rows')

close_prices_component_df = correlation_filter(close_prices_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X1=close_prices_component_df
vix=close_prices_df['^VIX']
vix_ret=close_prices_df['^VIX'].pct_change()
VIX=pd.DataFrame([vix, vix_ret]).T
VIX.columns=['VIX','VIX_ret']

X1=pd.concat([VIX,close_prices_component_df], axis=1)

In [411]:
# Inclusion of security returns X2
# Include returns that are correlated more than 0.20 with the Vix return

security_returns_df= close_prices_df.pct_change()
security_returns_component_df = correlation_filter(
                                        security_returns_df, 
                                        min_corr=0.20, 
                                        key_column='^VIX', 
                                        eliminate_first_column=True 
)

X2=security_returns_component_df


#### OBSERVATION:

In [412]:
# inclusion of security volume X3
volume_list = ticker_list[1:len(ticker_list)]

def retrieve_volume(volume_dict, volume_list):
    for ticker in volume_list:        
        volume = retrieve_yahoo_volume(ticker)
        volume_dict[ticker] = volume
    return volume_dict

if demo_mode == True:
    volume_df = pd.read_csv("adaboost_volume.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
else:
    volume_dict = {}
    volume_dict = retrieve_volume(volume_dict, volume_list)
    volume_df= pd.DataFrame(volume_dict)
    volume_df.to_csv("adaboost_volume.csv", index=True)
print("Completed retrieve of volume")

volume_df_with_vix=pd.concat([vix, volume_df], axis=1)
#print(volume_df_with_vix.corr())

volume_component_df = correlation_filter(volume_df_with_vix, min_corr=0.20, key_column='^VIX', eliminate_first_column=True )
X3=volume_component_df

Completed retrieve of volume


In [413]:
# Inclusion of GARCH series X4
garch_series=pd.DataFrame()

not_to_include=['^VIX']


for ticker in ticker_list:
    
        if ticker in not_to_include:
            continue
    
        if demo_mode == True:
            print_series = False
        else:
            print_series = True
        garch_series[ticker]=garch_fit_and_predict(security_returns_df[ticker], ticker, horizon=1, p=1, q=1, o=1, print_series_name=print_series)
            
        
X4=garch_series
if demo_mode == False:
    X4

Processing series: spy.....
Processing series: XLF.....
Processing series: XLE.....
Processing series: EURUSD=X.....
Processing series: GBPUSD=X.....
Processing series: AUDUSD=X.....
Processing series: BRLUSD=X.....
Processing series: DX-Y.NYB.....
Processing series: USDJPY=X.....
Processing series: ^TNX.....
Processing series: ZB=F.....
Processing series: ZF=F.....
Processing series: NQ=F.....
Processing series: NKD=F.....
Processing series: LQD.....
Processing series: AAPL.....
Processing series: AMZN.....
Processing series: GE.....
Processing series: MU.....
Processing series: MSFT.....
Processing series: BMY.....
Processing series: FDX.....
Processing series: GS.....
Processing series: PLD.....
Processing series: NVDA.....
Processing series: tlt.....
Processing series: ief.....
Processing series: FXI.....
Processing series: EZU.....
Processing series: EEM.....
Processing series: EFA.....
Processing series: FEZ.....
Processing series: ^GDAXI.....
Processing series: ^FTSE.....
Proces

In [414]:
# Inclusion of return squares in X5

returns_squared_df_no_vix= security_returns_df.drop(columns='^VIX')**2
returns_squared_and_vix_level_df=pd.concat([vix,returns_squared_df_no_vix], axis=1)
returns_squared_component_df = correlation_filter(returns_squared_and_vix_level_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X5=returns_squared_component_df
if demo_mode == False:
    X5

### Inclusion of Google Trends

In [415]:
# Upload of Google Tremds -- X6
keywords=['banking', "consumer", "depression", "gdp", "inflation",
          'unemployment', 'liquidity','cci', 'vix_word'] #'jobless_claims_pao', 
google_trends_df=pd.DataFrame()

for keyword in keywords:
    file_path=f"./Resources/Google_trends/{keyword}.csv"
    if demo_mode == False:
        print(file_path)
    trend=pd.read_csv(Path(file_path),
                      index_col= 'Daily', 
                      parse_dates= True,
                      infer_datetime_format=True
                     )
    #print(trend)
    google_trends_df=pd.concat([google_trends_df, trend], axis=1)
    #print(google_trends_df)

if demo_mode == False:
    google_trends_df

In [416]:
# Working on preparing Google-trends data

# Unifying google dates with VIX
minimum_date=vix.index.min()
maximum_date=vix.index.max()

google_trends_df=google_trends_df.loc[minimum_date:maximum_date,:]
#print(google_trends_df.iloc[0,:])

vix_google_trends_df=pd.concat([vix, google_trends_df], axis=1)
vix_google_trends_df.isna().sum()

#print(vix_google_trends_df.head())

#vix_google_trends_df=vix_google_trends_df.fillna(0)
#vix_google_trends_df
#vix_google_trends_df.loc[vix_google_trends_df['^VIX'].isna(),['^VIX','Banking: (United States)']]

# We will drop Saturday Sunday, but we would like to average Fri-Sat-Sun and reset the value of Friday
vix_google_trends_df=vix_google_trends_df.dropna()
google_trends_df=vix_google_trends_df.iloc[:,1:]
#google_trends_df.isna().sum()

In [417]:
# Filtering by correlation X6

google_trends_component_df = correlation_filter(
                                vix_google_trends_df, 
                                min_corr=0.05, 
                                key_column='^VIX', 
                                eliminate_first_column=True)

X6=google_trends_component_df

# We will interpolate so we can fill the missing data only on Google Trends
pro_interpolation_of_X6=pd.concat([vix, X6], axis=1)
pro_interpolation_of_X6=pro_interpolation_of_X6.interpolate(method="polynomial", order=2, axis=0)
pro_interpolation_of_X6
X6 = pro_interpolation_of_X6.iloc[:,1:]
if demo_mode == False:
    X6.shape

# Inclusion of Economic and Financial Series X7

In [418]:
#Economic Series
# Upload of csv files -- X7
keywords=['JobClaimsWeeklySeries', 'vix_put_call_ratio']
economic_series_df=pd.DataFrame()

for keyword in keywords:
    file_path=f"./Resources/Economic_and_financial_Series/{keyword}.csv"
    if demo_mode == False:
        print(file_path)
    new_series=pd.read_csv(Path(file_path),
                      index_col= 'DATE', 
                      parse_dates= True,
                      infer_datetime_format=True
                     )
    if keyword=='JobClaimsWeeklySeries':
        new_series=new_series.shift(-1, freq='D')
    if demo_mode == False:
        print(new_series)
    # Adjustment due to weekend data. We are going to assign data on the weekends to Friday, since are going to be 
    # consider for the the prediction of Monday
    economic_series_df=pd.concat([economic_series_df, new_series], axis=1)
    #print(economic_series_df.tail())

economic_series_df
economic_series_change_df = economic_series_df.pct_change().add_suffix('_change')

if demo_mode == False:
    economic_series_df.loc[:,:].tail(20)

In [419]:
# Working on indexes available

# Changes of columns that are on a weekend - concat with vix to add week days
vix_economic= pd.concat([vix,economic_series_df,economic_series_change_df ],axis=1)
vix_economic['VIX Put/Call Ratio']= vix_economic['VIX Put/Call Ratio'].fillna(0)

# Applying interpolation to appropiate columns. Levels: interpolation, changes: zeros
vix_economic.loc[:,economic_series_df.columns]=vix_economic.loc[:,economic_series_df.columns].interpolate(method="polynomial", order=2, axis=0)
vix_economic.loc[:,economic_series_change_df.columns]=vix_economic.loc[:,economic_series_change_df.columns].fillna(0)

#print(vix_economic)

#Filtering for available dates
economic_series_ready_df = vix_economic.loc[minimum_date:maximum_date,:]
economic_series_ready_df = economic_series_ready_df.iloc[:,1:]

X7=economic_series_ready_df

In [420]:
# SPY volatility on varios tracks X8

if demo_mode == True:
    close_price_spy_df = pd.read_csv("adaboost_spy_data.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
else:
    close_price_spy_df = retrieve_yahoo_close('spy', start_date='2005-01-01', end_date='2021-10-02')
    close_price_spy_df.to_csv("adaboost_spy_data.csv", index=True)
spy_returns_df=close_price_spy_df.pct_change()

spy_volatility_30_days = spy_returns_df.rolling(window=30).std()
spy_volatility_60_days = spy_returns_df.rolling(window=60).std()
spy_volatility_90_days = spy_returns_df.rolling(window=90).std()
spy_volatility_260_days = spy_returns_df.rolling(window=260).std()
spy_volatility_360_days = spy_returns_df.rolling(window=360).std()
spy_volatility_10_days = spy_returns_df.rolling(window=10).std()
spy_volatility_20_days = spy_returns_df.rolling(window=20).std()
spy_volatility_180_days = spy_returns_df.rolling(window=180).std()
spy_volatility_200_days = spy_returns_df.rolling(window=200).std()
spy_volatility_120_days = spy_returns_df.rolling(window=120).std()

if demo_mode == False:
    display(spy_volatility_120_days.shape)
    display(security_returns_df['spy'].shape)

X8=pd.concat([security_returns_df['spy'],
              spy_volatility_30_days ,
              spy_volatility_60_days,
              spy_volatility_90_days,
              spy_volatility_260_days,
              spy_volatility_360_days,
              spy_volatility_10_days,
               spy_volatility_20_days,
               spy_volatility_180_days,
               spy_volatility_200_days,
              spy_volatility_120_days],
             axis=1
                )
X8=X8.loc[minimum_date:maximum_date,:]
if demo_mode == False:
    display(X8.shape)
X8=X8.ffill()
X8=X8.iloc[:,1:]
if demo_mode == False:
    X8.shape

In [421]:
# Inclusion of Technical Indicators
technical_indicators = pd.read_csv("technical_indicators.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
technical_indicators = technical_indicators.drop(columns=["vix close", "vix return", "mean"])
X9 = pd.concat([security_returns_df['spy'], technical_indicators], axis=1)
X9=X9.loc[minimum_date:maximum_date,:]
if demo_mode == False:
    display(X9.shape)
X9=X9.ffill()
X9=X9.iloc[:,1:]
if demo_mode == False:
    display(X9.shape)
# display(X9.isna().sum())

In [422]:
#Inclusion of day of week data
day_of_week = pd.read_csv("prophet_output_day_of_week.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
day_of_week = day_of_week.drop(columns=["y"])
X10 = pd.concat([security_returns_df['spy'], day_of_week], axis=1)
X10 = X10.loc[minimum_date:maximum_date,:]
if demo_mode == False:
    display(X10.shape)
X10 = X10.ffill()
X10 = X10.iloc[:,1:]
if demo_mode == False:
    display(X10.shape)
#display(X10.isna().sum())

In [423]:
#Inclusion of neural network data
predictions_train_test_df = pd.read_csv("neural_network_output.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
X11 = pd.concat([security_returns_df['spy'], predictions_train_test_df], axis=1)
X11 = X11.loc[minimum_date:maximum_date,:]
if demo_mode == False:
    display(X11.shape)
X11 = X11.ffill()
X11 = X11.fillna(0)
X11 = X11.iloc[:,1:]
if demo_mode == False:
    display(X11.shape)

# GENERATION OF THE FEATURE MATRIX **X**

In [424]:
# Concatenation of all sources of data
XY=pd.concat([X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11], axis=1)
if parameter_tuning_mode == True:
    print(XY.shape)

XY.dropna(subset = ['VIX', 'VIX_ret'])
if parameter_tuning_mode == True:
    print(XY.shape)

# Interpolation is not applied to numerical variables. We are just going to drop those.
print(f"XY.shape: {XY.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X1.shape: {X1.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X2.shape: {X2.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X3.shape: {X3.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X4.shape: {X4.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X5.shape: {X5.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X6.shape: {X6.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X7.shape: {X7.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X8.shape: {X8.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X9.shape: {X9.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X10.shape: {X10.shape}, {XY.index.min()}, {XY.index.max()} ")
print(f"X11.shape: {X11.shape}, {XY.index.min()}, {XY.index.max()} ")

#display(XY.isna().head(40))
#display(XY.isna().sum().tail(40))
#XY=XY.dropna()
if parameter_tuning_mode == True:
    XY.shape

XY.shape: (3980, 201), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X1.shape: (3980, 21), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X2.shape: (3980, 28), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X3.shape: (3980, 21), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X4.shape: (3979, 39), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X5.shape: (3980, 33), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X6.shape: (3980, 3), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X7.shape: (3980, 4), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X8.shape: (3980, 10), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X9.shape: (3980, 36), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X10.shape: (3980, 5), 2006-07-03 00:00:00, 2021-10-01 00:00:00 
X11.shape: (3980, 1), 2006-07-03 00:00:00, 2021-10-01 00:00:00 


In [425]:
# Set the Signal column
vix_ret=vix.pct_change()
XY["Signal"] = 0.0

XY.loc[(XY['VIX_ret'] >= threshold), 'Signal'] = 1

# # Generate the trading signals 1 (entry) or -1 (exit)
# # where 1 is when the ^VIX is greater than 3.6%.
# # where 0 is when the ^VIX  is less than 3.6%.
#for index, row in XY.iterrows():
#    if row["VIX_ret"] >= 0.036:
#        XY.loc[index, "Signal"] = 1.0

# Review the DataFrame
if parameter_tuning_mode == True:
    print(XY["Signal"].head())
    XY["Signal"].value_counts()
XY.loc[XY["Signal"]==1, 'VIX_ret']
#XY.shape  ## 3981
XY.loc['2007-07-13':,'VIX_ret']

2007-07-13   -0.025097
2007-07-16    0.029043
2007-07-17    0.002566
2007-07-18    0.023672
2007-07-19   -0.048125
                ...   
2021-09-27    0.056901
2021-09-28    0.239339
2021-09-29   -0.029677
2021-09-30    0.025709
2021-10-01   -0.088159
Freq: B, Name: VIX_ret, Length: 3711, dtype: float64

In [426]:
vix_ret=vix.pct_change()
vix_ret[vix_ret>=threshold].index
vix_ret.shape


# How many values of the vix we missed due to missing data on other series
compare=pd.concat([XY.loc[XY["Signal"]==1, 'VIX_ret'],vix_ret[vix_ret>=threshold] ], axis=1)
missing_dates=compare.loc[compare["VIX_ret"]!=compare["^VIX"]]
missing_dates=missing_dates.index
missing_dates
if parameter_tuning_mode == True:
    vix[missing_dates]

vix_ret

Date
2006-07-03         NaN
2006-07-04    0.000000
2006-07-05    0.084291
2006-07-06   -0.035336
2006-07-07    0.023443
                ...   
2021-09-27    0.056901
2021-09-28    0.239339
2021-09-29   -0.029677
2021-09-30    0.025709
2021-10-01   -0.088159
Name: ^VIX, Length: 3980, dtype: float64

In [427]:
# Define the target set y using the Signal column
y = XY["Signal"]
# Display a sample of y
if parameter_tuning_mode == True:
    y
#pd.concat([vix_ret,y], axis=1).head(20)

In [428]:
if parameter_tuning_mode == True:
    display(y.isna().sum())
    display(y.shape)
    display(XY.shape)
    display(XY.drop(columns=["Signal"]).isna().sum())
    display(XY.drop(columns=['Signal']).shift().isna().sum())
    display(XY.drop(columns=['Signal']).shift().dropna().shape)

In [429]:
XY_modified = XY.shift().dropna()
if parameter_tuning_mode == True:
    display(XY_modified.shape)

y = XY_modified["Signal"].shift(-1)

X = XY_modified

if parameter_tuning_mode == True:
    display(y.shape)
    display(X.shape)
    display(y.isna().sum())
    display(X.isna().sum())

In [430]:
# Review the features DataFrame
if parameter_tuning_mode == True:
    X.head()

In [431]:
if parameter_tuning_mode == True:
    X.corr()

In [432]:
def split_training_test_data(X, y):
    # Split the preprocessed data into a training and testing dataset
    # Assign the function a random_state equal to 1
    training_begin = X.index.min()
    training_end = X.index.min() + DateOffset(months=training_period_months)

    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]

    X_test = X.loc[training_end + DateOffset(days=1):]
    y_test = y.loc[training_end + DateOffset(days=1):]

    if parameter_tuning_mode == True:
        print(f"Training dates: {training_begin} to {training_end}")
        display(y_train.value_counts())
        display(y_test.shape)
        display(X_test.shape)
        display(X_train.shape)
        display(y_train.shape)
        display(X_train.tail(1))
        display(X_test.head(1))
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = split_training_test_data(X, y)

In [433]:
def standard_scale(X_train, X_test):
    # Create a StandardScaler instance
    scaler =  StandardScaler() # MinMaxScaler() #
 
    # Apply the scaler model to fit the X-train data
    X_scaler = scaler.fit(X_train)

    # Transform the X_train and X_test DataFrames using the X_scaler
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    if parameter_tuning_mode == True:
        display(X_train_scaled.shape)
        display(X_test_scaled.shape)
    return X_train_scaled, X_test_scaled
    
X_train_scaled, X_test_scaled = standard_scale(X_train, X_test)

In [434]:
def adaboost_pca(X_train, X_test):
    pca = PCA(n_components = num_pca_components)
    pca.fit(X_train)
    
    principal_components_train = pca.transform(X_train)
    principal_components_test  = pca.transform(X_test)
    
    pca_column_list = []
    for i in range(1, num_pca_components+1):
        pca_column_list.append(f"pca{i}")

    principal_components_train_test = np.concatenate((principal_components_train, principal_components_test), axis = 0)
    principal_components_train_test_df = pd.DataFrame(data = principal_components_train_test, columns = pca_column_list, index = X.index)
    if parameter_tuning_mode == True:
        display(sum(pca.explained_variance_ratio_))
        display(principal_components_train_test_df.shape)
        display(principal_components_train_test_df.head(5))
    return principal_components_train_test_df
principal_components_train_test_df = adaboost_pca(X_train_scaled, X_test_scaled)

In [435]:
# Shift the LAG Component by a certain amount
def create_pca_lag(principal_components_train_test_df, shift_amount):
    X_pc_lag = principal_components_train_test_df.iloc[:,0:(number_of_pca_lag_component_to_include-1)]
    if parameter_tuning_mode == True:
        display(X_pc_lag.shape)

    X_pc_lag.columns = ['pca1_lag1','pca2_lag1','pca3_lag1']
    X_pc_lag = X_pc_lag.shift(shift_amount)

    if parameter_tuning_mode == True:
        print(X_pc_lag)
        X_pc_lag.shape
    return X_pc_lag

# Shift the LAG components by 1
X_pca_lag1 = create_pca_lag(principal_components_train_test_df, 1)

In [436]:
# Shift the LAG components by 2    
X_pca_lag2 = create_pca_lag(principal_components_train_test_df, 2)

In [437]:
# Shift the LAG components by 3
X_pca_lag3 = create_pca_lag(principal_components_train_test_df, 3)

In [438]:
# Shift the LAG Components by 4
X_pca_lag4 = create_pca_lag(principal_components_train_test_df, 4)

In [439]:
# Shift the LAG Components by 5
X_pca_lag5 = create_pca_lag(principal_components_train_test_df, 5);

In [440]:
def concatenate_lags(X_pc_lag1, X_pc_lag2, X_pc_lag3, X_pc_lag4, X_pc_lag5):
    X_pc_lags=pd.concat([X_pc_lag1, 
                         X_pc_lag2, 
                         X_pc_lag3, 
                         X_pc_lag4, 
                         X_pc_lag5], 
                         axis=1
    )
    
    if parameter_tuning_mode == True:
        X_pc_lags.shape
    return X_pc_lags

X_pc_lags = concatenate_lags(X_pca_lag1, X_pca_lag2, X_pca_lag3, X_pca_lag4, X_pca_lag5)

In [441]:
def combine_train_test(X_train, X_test):
    X_combined = np.concatenate([X_train, X_test], axis = 0)
    X_combined = pd.DataFrame(data = X_combined, index=X.index)
    return X_combined

def concatenate_with_pca_lags(X_raw, X_pc_lags):
    # Concatenation of all sources of data. Elimination of NaN due to lag
    X_pc = pd.concat([X_raw, X_pc_lags], axis=1)

    if parameter_tuning_mode == True:
        print(X_pc.shape)
    return X_pc

X_scaled_df = combine_train_test(X_train_scaled, X_test_scaled)
X_pc = concatenate_with_pca_lags(X_scaled_df, X_pc_lags)

In [442]:
# Interpolation is not applied to numerical variables. We are just going to drop those.
if parameter_tuning_mode == True:
    print(f"principal_components_train_test_df.shape: {principal_components_train_test_df.shape}, {principal_components_train_test_df.index.min()}, {principal_components_train_test_df.index.max()} ")
    print(f"X_pc_lags.shape: {X_pc_lags.shape}, {X_pc_lags.index.min()}, {X_pc_lags.index.max()} ")
    print(f"X_pc.shape: {X_pc.shape}, {X_pc.index.min()}, {X_pc.index.max()} ")
    print(f"y.shape: {y.shape}, {y.index.min()}, {y.index.max()}")

In [443]:
def eliminate_nans_in_pca_data(X_pc, y):
    X_pc = X_pc[5:-1]
    y = y[5:-1]

    if parameter_tuning_mode == True:
        display(X_pc.shape)
        display(y.shape)
    return X_pc, y

X_pc, y = eliminate_nans_in_pca_data(X_pc, y)

In [444]:
#Redefinition of X and y
X = X_pc

if parameter_tuning_mode == True:
    print(X.shape)
    y.shape

In [445]:
# Split of data in train and test, applying function that respect time series order, which is defined above

X_train, y_train, X_test, y_test = split_training_test_data(X, y)
X_train_scaled, X_test_scaled = standard_scale(X_train, X_test)

In [446]:
# Setting unique columns namess for random over sample model
column_name_list = []
for i in range(0, len(X.columns)):
    column_name_list.append(f"f_{i}")
X_train_unique_columns = X_train.copy()
X_train_unique_columns.columns = column_name_list

In [447]:
# Random oversample was applied since depending on the Threshold (above which return we are predicting), the sample can get highly unbalanced
# For the case of threshold = 0

def random_over_sample(X_train, y_train):
    # Use RandomOverSampler to resample the dataset using random_state=1
    ros = RandomOverSampler(random_state = 1)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    if parameter_tuning_mode == True:
        display(y_train_resampled.value_counts())
    return X_train_resampled, y_train_resampled

X_train_resampled, y_train_resampled = random_over_sample(X_train_scaled, y_train)

In [448]:
# Instance AdaBoost
# Initiate the model instance
adaboost_model=AdaBoostClassifier(n_estimators=adaboost_estimators)
adaboost_model

AdaBoostClassifier(n_estimators=22)

In [449]:
if parameter_tuning_mode == True:
    display(X_train_resampled.shape)
    display(y_train_resampled.shape)
    display(X_test_scaled.shape)
    display(y_test.shape)

In [450]:
# Fit the model 
adaboost_model =adaboost_model.fit(X_train_resampled, y_train_resampled)

pred_adaboost=adaboost_model.predict(X_test_scaled)

In [451]:
if demo_mode == False and parameter_tuning_mode == True:
    display(np.any(np.isnan(y_test)))
    display(np.all(np.isfinite(y_test)))
    display(np.any(np.isnan(pred_adaboost)))
    display(np.all(np.isfinite(pred_adaboost)))
    display(y_test.shape)
    display(pred_adaboost.shape)

In [452]:
# Use a classification report to evaluate the model using the predictions and testing data
adaboost_report=classification_report(y_test, pred_adaboost)

# Print the classification report
print("         AdaBoost Classification Report")
print(adaboost_report)


         AdaBoost Classification Report
              precision    recall  f1-score   support

         0.0       0.63      0.59      0.61       543
         1.0       0.53      0.57      0.55       443

    accuracy                           0.58       986
   macro avg       0.58      0.58      0.58       986
weighted avg       0.58      0.58      0.58       986



# Profitability Analysis
The large size of VIX returns, makes that if possible to bet on it and track its performance, it'spossible to make huge compounding effect.
As can be seen below, the average size of daily returns are around 5%, with returns that can get as large as 115% in one day.

In this section we will compare the results of betting on the model applying an imaginary strategy of betting at the open of the day, and close the bet at the end of the day. We will assume that the approximately the open level will be very similar than the close level of the previous day.

In [646]:
# VIX size of returns (in absolute value and %)
vix_return_statistics=abs(100 * security_returns_df['^VIX']).describe()
display(vix_statistics)

count    3979.000000
mean        5.325115
std         6.011000
min         0.000000
25%         1.523817
50%         3.750796
75%         7.161985
max       115.597925
Name: ^VIX, dtype: float64

In [636]:
# Results comnparison

# TRAIN WINDOW 
fit_train= adaboost_model.predict(X_train_scaled)
fit_train_df= pd.DataFrame(fit_train, index=X_train.index)

fit_train_df.hvplot()

y_train_df=pd.DataFrame(y_train, index=X_train.index)
y_train_df

vix_returns_train_df=vix_ret[y_train_df.index.min():y_train_df.index.max()]
vix_returns_train_df = vix_returns_train_df[y_train_df.index]


results_train_df=pd.concat([vix_returns_train_df, y_train_df, fit_train_df], axis=1)

results_train_df.columns=['VIX Return', 'Correct Signal', 'Fit Signal']

predicted_return=results_train_df['VIX Return']*results_train_df['Fit Signal']
max_return=results_train_df['VIX Return']*results_train_df['Correct Signal']


results_train_df=pd.concat([results_train_df, predicted_return, target_return], axis=1)
results_train_df.columns=['VIX Return', 'Correct Signal', 'Predicted Signal', 'Fit Return', 'Max Return']


roi_train_df=(1+results_train_df[['VIX Return','Fit Return']]).cumprod()
roi_train_df.columns=['VROI', 'ROI Model (in sample)']

roi_train_plot=roi_train_df.hvplot(
                                            title="Return on one Dollar Invested using AdaBoost Model"
)

roi_train_plot

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [653]:
# Results comnparison

# TEST WINDOW RESULTS FOR THE DAILY BET STRATEGY

# Out-of-sample Predictions 
prediction_test= adaboost_model.predict(X_test_scaled)
prediction_test_df= pd.DataFrame(prediction_test, index=X_test.index)

# Out-of-sample signals (1s or 0s) based on actual returns of the VIX
y_test_df=pd.DataFrame(y_test, index=X_test.index)
y_test_df

# VIX returns in the test window
vix_returns_df=vix_ret[y_test_df.index.min():y_test_df.index.max()]
vix_returns_test_df = vix_returns_df[y_test_df.index]

# Combination of VIX Returns, signals and predictions
results_test_df=pd.concat([vix_returns_test_df, y_test_df, prediction_test_df], axis=1)
results_test_df.columns=['VIX Return', 'Correct Signal', 'Predicted Signal']

# Predicted returns
predicted_return=results_test_df['VIX Return']*results_test_df['Predicted Signal']
max_return=results_test_df['VIX Return']*results_test_df['Correct Signal']

# DataFrame with out-of-sample results for comparison
results_test_df=pd.concat([results_test_df, predicted_return, max_return], axis=1)
results_test_df.columns=['VIX Return', 'Correct Signal', 'Predicted Signal', 'Predicted Return', 'Max Return']

roi_test_df=(1+results_test_df[['VIX Return', 'Predicted Return']]).cumprod()
roi_test_df.columns=['ROI VIX', 'ROI Model (out of sample)']

#Plot with out of sample return on a 1 dollar investment for the VIX, and the daily bet strategy
roi_test_plot=roi_test_df.hvplot(
                         title='Out-Of-Sample Return for $1 Investment ',
                         ylabel= "ROI in dollars",
                         width=1000
                        )
roi_test_plot


:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [544]:
# Results in prediction of daily returns
min_return=threshold
results_test_for_plot_df=results_test_df[abs(results_test_df['VIX Return'])>min_return]*100

results_test_for_plot_df.hvplot(
                    y=['VIX Return', 'Predicted Return'],
                    title= "Out-of-sample predictions of VIX return",
                    width=1000,
                    ylabel='Daily Return (%)'


)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [484]:
#Histogram of returns out of sample
results_test_for_plot_df.hvplot.hist(
    ['VIX Return','Predicted Return'],
     title= "Out-of-sample VIX returns predictions"
)

:NdOverlay   [Element]
   :Histogram   [Predicted Return]   (Predicted Return_count)

In [654]:
# Analysis of the good and bad out-of-sample predictions
good_predictions=results_test_df[results_test_df['Correct Signal']==results_test_df['Predicted Signal']]

bad_predictions=results_test_df[results_test_df['Correct Signal']!=results_test_df['Predicted Signal']]


In [655]:
good=good_predictions.hvplot.hist(
    ['VIX Return','Predicted Return'],
     title= "Out-of-sample VIX Returns of Good Predictions"
)

bad=bad_predictions.hvplot.hist(
    ['VIX Return','Predicted Return'],
     title= "Out-of-sample VIX Returns on Bad Predictions"
)
good+bad

:Layout
   .NdOverlay.I  :NdOverlay   [Element]
      :Histogram   [Predicted Return]   (Predicted Return_count)
   .NdOverlay.II :NdOverlay   [Element]
      :Histogram   [Predicted Return]   (Predicted Return_count)

In [658]:
# Good prediction statistics
good_predictions[['VIX Return', 'Predicted Return']].describe()

,VIX Return,Predicted Return
count,573.000000,573.000000
mean,-0.002669,0.028015
std,0.097663,0.076334
min,-0.233735,-0.086849
25%,-0.054125,-0.000000
50%,-0.011236,-0.000000
75%,0.028294,0.027387
max,1.155979,1.155979


In [659]:
# Bad prediction statistics
bad_predictions[['VIX Return', 'Predicted Return']].describe()

,VIX Return,Predicted Return
count,413.000000,413.000000
mean,0.009943,-0.021019
std,0.090317,0.038909
min,-0.205029,-0.205029
25%,-0.035869,-0.031899
50%,-0.006221,-0.003880
75%,0.038278,0.000000
max,0.616420,0.281172


In [660]:
# Box Plot predictions for good and bad predictions
good_pred=good_predictions['Predicted Return']
bad_pred =bad_predictions['Predicted Return']

predictions_comparison_df=pd.concat([good_predictions['Predicted Return'],bad_predictions['Predicted Return']], axis=1, ignore_index=True )*100
predictions_comparison_df.hvplot(kind='box',
                                height=800,
                                ylabel='Return (%)',
                                #clabel=['Good Predictions', 'Bad Predictions'],
                                cmap=['blue','red'],
                                title="Out-of-sample Good and Bad Returns Resulting from Model Predictions Box Plots")





df = pd.DataFrame(np.random.randn(20), columns=['Value'])
df['Source'] = ['Preds'] *10 +['Real'] * 10
df['Item'] = ['item1'] *5 + ['item2']*5 + ['item1'] *5 + ['item2']*5
df.hvplot.box(y='Value', by=['Item', 'Source'])

:BoxWhisker   [Variable]   (value)

In [457]:
def prepare_features(XY, pca_components):
    XY_modified = XY.shift().dropna()
    y = XY_modified["Signal"].shift(-1)
    X = XY_modified
    pca = PCA(n_components = pca_components)
    principal_components = pca.fit_transform(X)
    
    pca_column_list = []
    for i in range(1, pca_components+1):
        pca_column_list.append(f"pca{i}")

    principal_components_train_test_df = pd.DataFrame(data = principal_components, columns = pca_column_list, index = XY_modified.index)
    X_pca_lag1 = create_pca_lag1(principal_components_train_test_df)
    X_pca_lag2 = create_pca_lag2(principal_components_train_test_df)
    X_pca_lag3 = create_pca_lag3(principal_components_train_test_df)
    X_pca_lag4 = create_pca_lag4(principal_components_train_test_df)
    X_pca_lag5 = create_pca_lag5(principal_components_train_test_df)
        
    X_pc_lags = concatenate_lags(X_pca_lag1, X_pca_lag2, X_pca_lag3, X_pca_lag4, X_pca_lag5)
    X_pc = concatenate_pca_with_lags(principal_components_train_test_df, X_pc_lags)
    X, y = eliminate_nans_in_pca_data(X_pc, y)
    
    X_train, y_train, X_test, y_test = split_training_test_data(X, y)
    X_train_resampled, y_train_resampled = random_over_sample(X_train, y_train)
    X_train_scaled, X_test_scaled = standard_scale(X_train_resampled, X_test)
    #principal_components_train_test
    if parameter_tuning_mode == True:
        display(sum(pca.explained_variance_ratio_))
        display(principal_components_train_test_df.shape)
    return X_train_scaled, X_test_scaled, y_train_resampled, y_test

In [458]:
if run_multiple_tuning_iterations == True: 
    for num_estimators in range (20,200, 2):
        adaboost_model = AdaBoostClassifier(n_estimators=num_estimators)

        # Fit the model 
        adaboost_model = adaboost_model.fit(X_train_resampled, y_train_resampled)
        pred_adaboost = adaboost_model.predict(X_test)
        # Use a classification report to evaluate the model using the predictions and testing data
        adaboost_report=classification_report(y_test, pred_adaboost)

        #if num_estimators % 10 == 0 and num_components == 88:
        #    print(f"components {num_components} esimators {num_estimators}")
        #    print(f"f1 score 0 {f1_score(y_test, pred_adaboost, pos_label=0)} f1 score 1 {f1_score(y_test, pred_adaboost, pos_label=1)}")
        #    print(f"accuracy {accuracy_score(y_test, pred_adaboost)}")
        #    print(adaboost_report)
        f1_score_1 = f1_score(y_test, pred_adaboost, pos_label=1)
        f1_score_0 = f1_score(y_test, pred_adaboost, pos_label=0)
        recall_score_1 = recall_score(y_test, pred_adaboost, pos_label=1)
        recall_score_0 = recall_score(y_test, pred_adaboost, pos_label=0)
        accuracy_score_model = accuracy_score(y_test, pred_adaboost)
        if  accuracy_score_model >= .55 and f1_score_1 >= .50 and f1_score_0 >= .50 and recall_score_1 >= .50 and recall_score_0 >= .50:
            print(f"estimators {num_estimators}")
            # print(f"variance explained {sum(pca.explained_variance_ratio_)}")
            # Print the classification report
            print("         AdaBoost Classification Report")
            print(adaboost_report)

In [338]:
# Number of estimators? 
if run_multiple_tuning_iterations == True:
    for n in range (50,200, 10):
        # Instance AdaBoost
        # Initiate the model instance
        adaboost_model=AdaBoostClassifier(n_estimators=n)

        # Fit the model 
        adaboost_model =adaboost_model.fit(X_train_scaled, y_train)
        pred_adaboost=adaboost_model.predict(X_test_scaled)
        print (n)
        # Use a classification report to evaluate the model using the predictions and testing data
        adaboost_report=classification_report(y_test, pred_adaboost)

        # Print the classification report
        print("         AdaBoost Classification Report")
        print(adaboost_report)
#120 highest 1-recall
#150 good overall accuracy, but lower 1-recall


#### Future enhancements:

* Add VIX and VIX return in the correlation_filter 

* Manage names in the features: spy_volume, spy_return, spy_close, etc

* To include VIX level and Return and check on both in case correlations changes in the future. As of now with some variables I check with VIX level, and others I check VIX return based on the historical correlations